In [41]:
import numpy as np
import pandas as pd
import pyspark as ps
import matplotlib.pyplot as plt
import csv

import unicodedata

from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from gensim import corpora, models

import pyLDAvis.gensim

/Users/lguillain/anaconda/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
df = pd.read_json('reduced_tweets.json', lines=True)

## Necessary data processing:

Different languages have different pattern but no difference betweet mental illness related tweets and unrelated tweets can be found

In [ ]:
#this might be a dumb idea in retrospect. we need to keep èä to use stop word removal effectivley
# df['main'] = df['main'].astype(str).str.lower().\
#                     apply(lambda tweet: unicodedata.normalize('NFD', tweet).\
#                     encode('ascii', 'ignore').decode('utf-8'))
# df['main'] = df['main'].str.replace(r'[^\w\s]', '')
# df['main'].head()

Why we should use the tweets tokenizer:

In [11]:
tknzr = TweetTokenizer()
tknzr.tokenize('mir gahts so so lala lol!! merd:X :D')

['mir', 'gahts', 'so', 'so', 'lala', 'lol', '!', '!', 'merd', ':', 'X', ':D']

In [10]:
'mir gahts so so lala lol!! :D'.split()

['mir', 'gahts', 'so', 'so', 'lala', 'lol!!', ':D']

In [12]:
tokenized = df['main'].map(lambda x: tknzr.tokenize(x))

In [13]:
#tokenized.to_pickle('tokenized_strings.pkl')
#save time and read from this pkl

In [14]:
# df['split'] = df['main'].str.split()
df['tokenized'] = tokenized

In [16]:
def remove_stops(language, cleaned, frame):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    lang_set = stopwords.words(language)
    cleaned.loc[cleaned['lang'] == lang, frame] = cleaned.loc[cleaned['lang'] == lang, frame].\
        apply(lambda tweet: [word for word in tweet if word not in lang_set])

In [17]:
remove_stops('english', df, 'tokenized')
remove_stops('french', df, 'tokenized')
remove_stops('german', df, 'tokenized')

In [18]:
df.head()

,author_gender,lang,main,published,sentiment,source_followers,source_following,source_location,source_spam_probability,tags,tokenized
0,FEMALE,fr,en espérant que 2016 soit meilleur que 2015 qu...,2016-01-01T00:30:04Z,NEUTRAL,80,140,Saint-Maurice,0,NaN,"[espérant, 2016, meilleur, 2015, quand]"
1,UNKNOWN,en,nice bmw ...he kills ant s!!!,2016-01-01T12:12:33Z,NEUTRAL,79,196,Hinwil,0,NaN,"[nice, bmw, ..., kills, ant, !, !, !]"
2,MALE,de,"ich hab nichts dagegen, wenn mir andere beim...",2016-01-01T19:52:15Z,NEUTRAL,479,783,Schenkon,0,NaN,"[dagegen, ,, beim, saufen, zuschauen, ,, solan..."
3,UNKNOWN,en,happy new jear!!!! i am back from 2 weeks cuba...,2016-01-01T11:58:03Z,NEUTRAL,79,196,Hinwil,0,NaN,"[happy, new, jear, !, !, !, back, 2, weeks, cu..."
4,UNKNOWN,fr,bonne année à toi je prends beaucoup de plai...,2016-01-01T06:17:28Z,NEUTRAL,75,984,Bâle,0,NaN,"[bonne, année, prends, beaucoup, plaisir, suiv..."


In [19]:
def stem_words(language, cleaned, frame):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    stemmer = SnowballStemmer(language)
        
    cleaned.loc[cleaned['lang'] == lang, frame] = cleaned.loc[cleaned['lang'] == lang, frame].\
        apply(lambda tweet: [stemmer.stem(word) for word in tweet])

In [20]:
stem_words('english', df, 'tokenized')
stem_words('french', df, 'tokenized')
stem_words('german', df, 'tokenized')

In [21]:
df.tokenized.to_pickle('tokenized_stemmed.pkl')

In [22]:
DICT_PATH = "dictionary.csv"
dictionaries = pd.read_csv(DICT_PATH)
dictionaries.head()

,english,french,german,swiss_german
0,addict,accro,sucht,NaN
1,afraid,peur,angst,angscht
2,agoraphobia,agoraphobie,NaN,NaN
3,alcoholism,alcoolisme,alcoholismus,NaN
4,alone,seul,allein,allei


In [23]:
en_dict = dictionaries['english'].dropna()
fr_dict = dictionaries['french'].dropna()
de_dict = pd.concat([dictionaries['german'].dropna(), dictionaries['swiss_german'].dropna()])

In [24]:
def dict_cleaning(lang):
    lang_dict = eval(lang + '_dict')
    lang_dict = lang_dict.astype(str).str.lower().\
                        apply(lambda expression: unicodedata.normalize('NFD', expression).\
                        encode('ascii', 'ignore').decode('utf-8'))

In [25]:
dict_cleaning('en')
dict_cleaning('fr')
dict_cleaning('de')

In [26]:
#Tokenizing
en_dict = en_dict.str.split()
fr_dict = fr_dict.str.split()
de_dict = de_dict.str.split()

In [27]:
def dict_remove_stops(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_dict = eval(lang + '_dict')
    lang_set = stopwords.words(language)
        
    lang_dict = lang_dict.apply(lambda expression: [word for word in expression if word not in lang_set])

In [28]:
#Removing stop words
dict_remove_stops('english')
dict_remove_stops('french')
dict_remove_stops('german')

In [29]:
def dict_stem_words(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_dict = eval(lang + '_dict')
    stemmer = SnowballStemmer(language)
        
    lang_dict = lang_dict.apply(lambda expression: [stemmer.stem(word) for word in expression])

In [30]:
#Stemming the words
dict_stem_words('english')
dict_stem_words('french')
dict_stem_words('german')

In [31]:
df['merged'] = df.tokenized.map(lambda x:  ' '.join(x))
de_dict = de_dict.map(lambda x:  ' '.join(x))
en_dict = en_dict.map(lambda x:  ' '.join(x))
fr_dict = fr_dict.map(lambda x:  ' '.join(x))

In [32]:
def check_dict(tweet, dict_):
    """checks if dict entry matches tweet"""
    match = [ w for w in dict_ if w in tweet] #find matching for each entry
    return len(match) > 0 #at least one match

In [33]:
english_filtered = df[df.lang == 'en'][df[df.lang == 'en']['merged'].map(lambda x: check_dict(x, en_dict))]

,author_gender,lang,main,published,sentiment,source_followers,source_following,source_location,source_spam_probability,tags,tokenized,merged
210,UNKNOWN,en,i feel sad that they have to hide and probab...,2016-01-01T15:38:09Z,NEGATIVE,598,316,Svizzera,0,NaN,"[feel, sad, hide, probabl, won't, ever, abl, g...",feel sad hide probabl won't ever abl go fun li...
316,UNKNOWN,en,"i hope people will react positively, but i'm...",2016-01-01T12:41:23Z,NEUTRAL,598,316,Svizzera,0,NaN,"[hope, peopl, react, posit, ,, i'm, afraid, fa...","hope peopl react posit , i'm afraid fangirl"
349,UNKNOWN,en,but hani's dating news makes me feel sad... be...,2016-01-01T15:36:24Z,NEUTRAL,598,316,Svizzera,0,NaN,"[hani, date, news, make, feel, sad, ..., it, n...",hani date news make feel sad ... it normal ido...
617,MALE,en,". ah yes, there he is about 30 mins in. not...",2016-01-01T12:31:56Z,NEUTRAL,600,850,Bern,0,NaN,"[., ah, yes, ,, 30, min, ., sure, spurious, fi...",". ah yes , 30 min . sure spurious first person..."
2108,FEMALE,en,homesickness again...,2016-01-01T23:23:53Z,NEUTRAL,1393,1921,Zürich,0,NaN,"[homesick, ...]",homesick ...


In [39]:
english_filtered.main.sample(5)

3365506      haha thank you... i'm a little sad that all ...
2896805    wikipedian of the year   adressing #gendergap ...
590069     nearshore team requests sysadmin rights to "de...
3401235    it makes me so sad and mad knowing that two gi...
220045     layan runningman cara perempuan hilangkan stre...
Name: main, dtype: object

In [34]:
english_filtered.count()

author_gender              20818
lang                       20818
main                       20818
published                  20818
sentiment                  20818
source_followers           20818
source_following           20818
source_location            20191
source_spam_probability    20818
tags                        4766
tokenized                  20818
merged                     20818
dtype: int64

In [100]:
french_filtered = df[df.lang == 'fr'][df[df.lang == 'fr']['tokenized'].map(lambda x: check_dict(x, fr_dict))]
french_filtered.head()

,author_gender,lang,main,published,sentiment,source_followers,source_following,source_location,source_spam_probability,tags,tokenized,merged
330,UNKNOWN,fr,suis je le seul debout ?,2016-01-01T11:06:40Z,NEUTRAL,276,241,Fribourg,0,NaN,"[seul, debout, ?]",seul debout ?
343,UNKNOWN,fr,je crois je vais allez regarder les feu d'arti...,2016-01-01T02:19:52Z,NEUTRAL,275,632,Geneve,0,NaN,"[crois, vais, allez, regard, le, feu, d'artifi...",crois vais allez regard le feu d'artific tout ...
669,UNKNOWN,fr,je rigole encore tout seul dans ma chambre d'hier,2016-01-01T14:08:50Z,NEUTRAL,128,93,Lausanne,0,NaN,"[rigol, encor, tout, seul, chambr, d'hi]",rigol encor tout seul chambr d'hi
686,UNKNOWN,fr,vécu. c'est sûrement la seule chose de ma vi...,2016-01-01T09:42:47Z,NEUTRAL,327,289,Lausanne,0,NaN,"[vécu, ., c'est, sûr, seul, chos, vi, sujet, l...",vécu . c'est sûr seul chos vi sujet laquel cer...
691,UNKNOWN,fr,pas mais qui dureront aussi longtemps que no...,2016-01-01T09:34:43Z,NEUTRAL,327,289,Lausanne,0,NaN,"[dur, auss, longtemp, ., j'en, auss, ., le, ta...",dur auss longtemp . j'en auss . le tais . peur...


In [102]:
len(french_filtered)

20533

In [101]:
german_filtered = df[df.lang == 'de'][df[df.lang == 'de']['tokenized'].map(lambda x: check_dict(x, de_dict))]
german_filtered.head()

,author_gender,lang,main,published,sentiment,source_followers,source_following,source_location,source_spam_probability,tags,tokenized,merged
65,MALE,de,"rtl bringt das kunststück fertig, eine klassis...",2016-01-01T19:35:15Z,NEUTRAL,479,783,Schenkon,0,NaN,"[rtl, bringt, kunststuck, fertig, ,, klassisch...","rtl bringt kunststuck fertig , klassisch pleit..."
119,MALE,de,"danket dem herrn aller herren, der allein groß...",2016-01-01T08:43:21Z,NEUTRAL,1101,1963,Chur,0,NaN,"[danket, herrn, herr, ,, allein, gross, wund, ...","danket herrn herr , allein gross wund tut , gu..."
558,UNKNOWN,de,"nei nei, keine angst. ich reise nicht mehr g...",2016-01-01T21:53:17Z,NEUTRAL,38400,36900,CH,0,NaN,"[nei, nei, ,, angst, ., reis, mehr, gern, ...]","nei nei , angst . reis mehr gern ..."
624,FEMALE,de,"ja das gibts bestimmt. aber auch die, welche...",2016-01-01T13:36:32Z,NEUTRAL,459,450,Bern,0,NaN,"[ja, gibt, bestimmt, ., ,, wirklich, "", allein...","ja gibt bestimmt . , wirklich "" allein stehend..."
786,UNKNOWN,de,aber es geht ihr wieder besser.. trotzdem. sie...,2016-01-01T21:04:23Z,NEUTRAL,20,53,Bern,0,NaN,"[geht, bess, .., trotzd, ., nie, alt, &, macht...",geht bess .. trotzd . nie alt & macht traurig .


In [103]:
len(german_filtered)

9775

In [ ]:
#english_filtered.to_csv('english.csv')

In [ ]:
#english_filtered.sample(n=1000).to_csv('english_1000.csv')

In [ ]:
#french_filtered.sample(n=1000).to_csv('french_1000.csv')

In [ ]:
#move this above
import re
s = "string. With. Punctuation? ! àéè. : ; "
s = re.sub(r'[^\w\s]','',s)

In [69]:
english_filtered_2 = english_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

## LDA starts here

In [71]:
dictionary = corpora.Dictionary(english_filtered_2)

In [72]:
dictionary.filter_extremes(no_below=3, no_above=.5)

In [73]:
print(dictionary)

Dictionary(5895 unique tokens: ['feel', 'sad', 'hide', 'probabl', 'wont']...)


In [74]:
texts = english_filtered_2.tolist()

In [75]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [93]:
ldamodel = models.LdaMulticore(corpus, id2word=dictionary, num_topics=3, workers=3, iterations=100, passes=5) #takes like 5minutes on leo's pc

In [94]:
ldamodel.show_topics()

[(0,
  '0.027*"assad" + 0.022*"syria" + 0.019*"stress" + 0.010*"ambassador" + 0.007*"afraid" + 0.007*"stop" + 0.007*"kill" + 0.006*"russia" + 0.006*"asylum" + 0.005*"addict"'),
 (1,
  '0.094*"sad" + 0.017*"im" + 0.014*"it" + 0.012*"make" + 0.011*"dont" + 0.009*"know" + 0.009*"day" + 0.009*"peopl" + 0.009*"afraid" + 0.007*"time"'),
 (2,
  '0.042*"sad" + 0.031*"im" + 0.019*"stress" + 0.015*"addict" + 0.014*"afraid" + 0.008*"one" + 0.007*"get" + 0.007*"see" + 0.007*"go" + 0.006*"u"')]

In [88]:
#ldamodel.save('lda.model') #save lda model

In [ ]:
#ldamodel =  models.LdaModel.load('lda.model') #retrive lda model

In [96]:
vis_data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/lguillain/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [97]:
pyLDAvis.display(vis_data)

## Same for french etc:

In [104]:
french_filtered_2 = french_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

In [108]:
french_filtered_2.head()

330                                     [seul, debout, ]
343    [crois, vais, allez, regard, le, feu, dartific...
669              [rigol, encor, tout, seul, chambr, dhi]
686    [vécu, , cest, sûr, seul, chos, vi, sujet, laq...
691    [dur, auss, longtemp, , jen, auss, , le, tais,...
Name: tokenized, dtype: object

In [111]:
fr_dictionary = corpora.Dictionary(french_filtered_2)
fr_dictionary.filter_extremes(no_below=5, no_above=.3)
fr_texts = french_filtered_2.tolist()
fr_corpus = [fr_dictionary.doc2bow(text) for text in fr_texts]

ldamodel_2 = models.LdaMulticore(fr_corpus, id2word=fr_dictionary, num_topics=10, workers=3, iterations=100, passes=5)

vis_data = pyLDAvis.gensim.prepare(ldamodel_2, fr_corpus, fr_dictionary)

pyLDAvis.display(vis_data)

/Users/lguillain/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


## Same for german:

In [105]:
german_filtered_2 = german_filtered.tokenized.map(lambda x: [re.sub(r'[^\w\s]','',s) for s in x])

In [116]:
ger_dictionary = corpora.Dictionary(german_filtered_2)
ger_dictionary.filter_extremes(no_below=3, no_above=.5)
ger_texts = german_filtered_2.tolist()
ger_corpus = [ger_dictionary.doc2bow(text) for text in ger_texts]

ldamodel_2 = models.LdaMulticore(ger_corpus, id2word=ger_dictionary, num_topics=5, workers=3, iterations=100, passes=5)

vis_data = pyLDAvis.gensim.prepare(ldamodel_2, ger_corpus, ger_dictionary)

pyLDAvis.display(vis_data)

/Users/lguillain/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]
